In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (13, 10)
import math
from pprint import pprint

In [14]:
#  As I see it, we've got 6 options
# 1A) 9s1p Battery just for the TVC
# 1B) Ns1p TVC battery with minimum capacity to run the burn
# 2A) 2sNp where N is the minimum number of strings (lightest Dove extended)
# 2B) 2s7p, slightly lighter than 2s3p stock Dove + 9s1p
# 3A) 3sNp minimizing N, a little bit lower current, but new system voltage
# 3B) 3s5p, same number of cells, lower current, new system voltage

#  There are many subjective considerations, for system simplicity, reliability, robustness...
# but here I'm just trying to characterize the objective tradeoffs, I guess mass and thermal
# mostly. Also, I'm assuming that 3sNp is useable by the main avionics; if it's not then that's
# out the window clearly.

#  Requirements are to provide the total energy needed to run the TVC gimbal for a 10 minute,
# a limited (~10) number of times, and be able to provide the peak power required (current
# estimate: 200W). In the case of only one system battery pack, I'll just assume that there's
# some overhead energy requirement that is small relative to the TVC gimbal requirements and
# if it's large enough to matter we'd just add one more parallel string to deal with it. In
# the absence of an actual measure for total burn energy, I'm going to assume max power the
# whole time. Also assuming that voltage conversion is 90% power-efficient, needed if the
# power to the gimbal cannot be guaranteed to be within 8-36V (especially on the low end).

Peff = 0.9           # of voltage conversion
Ppk  = 200.          # [W] max draw
Etot = 200. * 10./60 # [Wh] energy for a burn

# some battery specs, roughly
tcell  = 'NCR18650B'
Rint   = 0.06 # [Ohm]
Vmax   = 4.1  # [V]
SOCmax = 3.   # [Ah]
Vmin   = 3.4  # [V] under no charge/discharge current
SOCmin = 0.25 # [Ah]
C      = 3.2  # [A]
Imax   = 4.8  # [A]

if True:
    tcell = 'NCR18650PF'
    SOCmax = 2.5
    C = 2.7
    Imax = 10

# sanity checks
print('Working with {} cells'.format(tcell))
cell_cap = (Vmax + Vmin) / 2 * (SOCmax - SOCmin)
print('available capacity: {:.2f} Wh/cell'.format(cell_cap))
print('theoretical energy-limited minimum cell count: {:.1f}'.format(Etot / cell_cap))
print('current-limited minimum cell count: {:.1f}'.format(Ppk / (Imax * Vmin)))

Working with NCR18650PF cells
available capacity: 8.44 Wh/cell
theoretical energy-limited minimum cell count: 4.0
current-limited minimum cell count: 5.9


In [58]:
def roots(a, b, c, allowImag=False):
    imag = False
    foo = b * b - 4 * a * c
    if foo < 0:
        if not allowImag:
            raise ValueError('trying to take the sqrt of a negative')
        imag = True
        foo = 0 - foo
    r = (((0 - b) - math.sqrt(foo)) / (2 * a),
         ((0 - b) + math.sqrt(foo)) / (2 * a))
    if imag:
        r = (r[0] * 1j, r[1] * 1j)
    return r

def imax(ns, np, power):
    videal = ns * Vmin
    r = (Rint * ns) / np
    # 0 = -r * I * I + videal * I - power
    maxes = roots(0-r, videal, 0-power)
    return min([m for m in maxes if m >= 0])

# What do 2s4p and 3s3p packs look like?
print(imax(3, 3, 200))
print(imax(2, 4, 200))
print(imax(8, 1, 200))

22.61677575929033
34.7344925163227
8.683623129080676


In [44]:
def report(ns, np, power):
    print('{}s{}p: \t{:5.1f}Wh \t{:5.2f}A max \t{}g'.format(ns, np,
                                                            ns * np * cell_cap,
                                                            imax(ns, np, power) / np,
                                                            ns * np * 50
                                                           ))

# ok then, let's check out options. Shortest Ns1p that could work?
for ns in range(1, 10):
    try:
        if imax(ns, 1, 200) <= Imax:
            report(ns, 1, 200)
            break
    except:
        pass
    
# how about a 10s1p, the longest that is practical (and only marginally so)?
report(10, 1, 200)

# what does a 2s3p (stock Dove) look like? Bumping up power because we need DC-DC
report(2, 3, 220)
# oh, too demanding. How about 2s4p?
report(2, 4, 220)
# barely within limits, ok then 2s5p?
report(2, 5, 220)

# what if we can go up to 3s3p? possibly still manageable by stock avionics, no DC-DC
report(3, 2, 200)
report(3, 3, 200)

8s1p: 	 67.5Wh 	 8.68A max 	400g
10s1p: 	 84.4Wh 	 6.67A max 	500g
2s3p: 	 50.6Wh 	14.49A max 	300g
2s4p: 	 67.5Wh 	 9.77A max 	400g
2s5p: 	 84.4Wh 	 7.45A max 	500g
3s2p: 	 50.6Wh 	12.61A max 	300g
3s3p: 	 75.9Wh 	 7.54A max 	450g


In [89]:
# to do a mass roll-up, what is system cell mass, tvc cell mass, tvc wiring?
cmass = 48
sharing = True
sysmass = 2 * 3 * cmass if not sharing else 0
wirelen = 2 if sharing else 0.5

           #AWG, ohms/1000ft, lbs/1000ft
ughwires = [(18, 6.386, 10.),
            (16, 4.019, 14.),
            (14, 2.524, 20.),
            (12, 1.589, 28.),
            (10, 0.9988, 42.),
            (8, 0.6281, 72.),
           ]
# convert to (AWG, ohms/m, g/m)
wires = [(w[0], w[1] / 1000 / 0.3, w[2] / 0.0022 / 1000 / 0.3)
         for w in ughwires]

# let's look at a 3s3p total mass, assume 2m wire
# 0 = 30*30*(Rint + Rcable) - (3 * Vmin)*30 + 200
# (3*Vmin*30 - 900*Rint - 200) / 900. = Rcable
Rc = (3*Vmin*30 - 900*Rint - 200) / 900.
print(Rc)
print('awg, current, wire resistance, battery waste, wire waste, wire mass, minutes run')
for (awg, r, m) in wires:
    if 2 * r > Rc:
        pass
    try:
        i = min([r for r in roots(0 - (2 * r) - Rint, 3 * Vmin, -200.)
                 if r > 0])
    
        print('{:2}  {:.2f}  {:4.1f}  {:.2f}  {:5.2f}  {:.1f}  {}'.format(awg, i, 1000*2*r, i*i*Rint, i*i*2*r, 2*m,
                                                                          int(9*cell_cap/(200 + i*i*(Rint + 2*r)) * 60)))
    except ValueError:
        pass
#current = imax(3, 3, 200)

0.057777777777777775
awg, current, wire resistance, battery waste, wire waste, wire mass, minutes run
18  26.87  42.6  43.31  30.73  30.3  16
16  24.87  26.8  37.12  16.57  42.4  17
14  23.92  16.8  34.32   9.62  60.6  18
12  23.40  10.6  32.84   5.80  84.8  19
10  23.09   6.7  32.00   3.55  127.3  19
 8  22.91   4.2  31.50   2.20  218.2  19


In [112]:
def system(ns, np, lwire=2, power=200, trun=0.2, msys=(48*6)):
    # can we handle max power with perfect wires?
    cell_imax = imax(ns, np, power) / np
    if cell_imax > Imax:
        raise Exception('too much peak current per cell')
    # what are our wiring options?
    wiring = []
    for (awg, r, m) in wires:
        I = min(roots(0 - (r * lwire) - Rint*ns/np, ns * Vmin, 0 - power, allowImag=True))
        # is that a real root?
        if I.imag != 0:
            continue
        # do we still respect max current per cell?
        if I / np > Imax:
            continue
        # do we have enough energy?
        # TODO I'm not sure geometric mean is correct here
        Ilo, Ihi = min(roots(0 - (r * lwire) - Rint*ns/np, ns*Vmax, 0 - power)), I
        Imean = (Ilo + Ihi) / 2.
        Emean = (Imean * Imean * (Rint * ns / np + r*lwire) + power)
        Eavail = ns * np * (0.5 * (Vmax + Vmin)) * (SOCmax - SOCmin)
        if Eavail < Emean * trun:
            continue
        wiring.append({'awg': awg,
                       'wire_resistance': r * lwire,
                       'wire_mass': m * lwire,
                       'max_run': Eavail / Emean,
                       'wire_diss': Imean * Imean * r * lwire,
                       'batt_diss': Imean * Imean * Rint * ns / np,
                       'tot_mass': msys + ns * np * cmass + m * lwire,
                       'Isys_max': Ihi
                      })
    return wiring

In [119]:
#pprint(sorted(system(9,1,1), key=lambda sys: sys['tot_mass']))
pprint(sorted(system(8,1,1), key=lambda sys: sys['tot_mass']))
pprint(sorted(system(3,3, msys=0), key=lambda sys: sys['tot_mass']))
#pprint(sorted(system(4,3, msys=0), key=lambda sys: sys['tot_mass']))
pprint(sorted(system(4,2, msys=0), key=lambda sys: sys['tot_mass']))

[{'Isys_max': 8.770619937501609,
  'awg': 18,
  'batt_diss': 29.113373185677037,
  'max_run': 0.2929630656824193,
  'tot_mass': 687.1515151515151,
  'wire_diss': 1.2910972303037054,
  'wire_mass': 15.15151515151515,
  'wire_resistance': 0.02128666666666667},
 {'Isys_max': 8.737921412244038,
  'awg': 16,
  'batt_diss': 28.939046109751974,
  'max_run': 0.2938017917645583,
  'tot_mass': 693.2121212121212,
  'wire_diss': 0.8076807382992583,
  'wire_mass': 21.21212121212121,
  'wire_resistance': 0.013396666666666668},
 {'Isys_max': 8.717546917874362,
  'awg': 14,
  'batt_diss': 28.830478640839623,
  'max_run': 0.2943282130999831,
  'tot_mass': 702.3030303030303,
  'wire_diss': 0.5053342228436057,
  'wire_mass': 30.3030303030303,
  'wire_resistance': 0.008413333333333335},
 {'Isys_max': 8.704911345789776,
  'awg': 12,
  'batt_diss': 28.7631708544242,
  'max_run': 0.2946561623880134,
  'tot_mass': 714.4242424242424,
  'wire_diss': 0.31739360060888927,
  'wire_mass': 42.42424242424242,
  'wire